# Clustering

Type of clustering algorithms:
- hard clustering (k-means): clusters do not overlap
- soft clustering (EM): clusters may overlap with probabilities


## K-Means

Objective: Given $D = \{ x_1, x_2, ..., x_N \}$, find k clsuters $C = \{C_1, C_2, ..., C_k \}$ to minimize

$$\min_{C} \sum_{k=1}^K \sum_{x_i \in C_k} ||x_i - \mu_k||^2$$
$$\mu_k = \frac{1}{|C_k|} \sum_{x_i \in C_k} x_i$$

Steps:
- initialize k random centroids $\{ \mu_1, \mu_2, ..., \mu_k \}$
- for each data point $\{ x_1, x_2, ..., x_N \}$, find the closest centrod
- for each cluster $\{C_1, C_2, ..., C_k \}$, recompute centroid

In [1]:
import numpy as np

def compute_dist(x1, x2):
    return sum((x1 - x2) * (x1 - x2))

def kmeans(X, k, num_iter=100):
    """
    Implement kmeans algorithm
    @X: numpy array
    @k: int (number of clusters)
    @num_iter: int (number of iterations)
    """
    # choose random datapoint as initial centroids
    index = np.random.choice(len(X), size=k, replace=False)
    centroids = X[index, :]
    
    for _ in range(num_iter):
        distances = []
        
        # compute distance between point and each centroid
        for i in range(k):
            distance = [compute_dist(x, centroids[i]) for x in X]
            distances.append(distance)
        
        # find the closest centrod for each data point
        min_distance = np.array([np.argmin(x) for x in np.array(distances).T])
        
        # compute new centroids
        new_centroids = []
        for i in range(k):
            centroid = X[min_distance==i, :].mean(axis=0)
            # print(centroid)
            new_centroids.append(centroid)
        centroids = np.array(new_centroids)
        
    return min_distance, centroids

In [2]:
kmeans(np.random.random((50, 2)), 5)

(array([1, 1, 2, 1, 1, 4, 4, 1, 4, 1, 4, 3, 4, 1, 0, 4, 2, 2, 2, 2, 1, 0,
        2, 2, 1, 2, 2, 1, 0, 3, 1, 3, 0, 4, 1, 0, 3, 1, 3, 2, 1, 4, 0, 0,
        3, 2, 1, 2, 2, 0]),
 array([[0.90634388, 0.56382264],
        [0.24078613, 0.33989064],
        [0.70170277, 0.83013909],
        [0.09067113, 0.82331057],
        [0.65647735, 0.20463219]]))

# Expectation Maximization (EM)

Objective: Let $Y = \{ y_1, y_2, ..., y_N \}$ be observed variable, $Z = \{ z_1, z_2, ..., z_N \}$ be hidden variable, maximize likelihood 

$$L(\theta) = \sum_{i=1}^N log p(y_i; \theta) = \sum_{i=1}^N log (\sum_z p(y_i, z; \theta)) = \sum_{i=1}^N log (\sum_z p(y_i | z; \theta)p(z; \theta))$$

Steps:
- start with randomly placed parameters
- E step: compute expectation based on posterior probability

$$Q(\theta, \theta^{(i)}) = \sum_{j=1}^N E_{P(Z|y_k; \theta^{(i)})} log P(y_j, Z; \theta) = \sum_{j=1}^N (\sum_Z P(Z|y_j; \theta^{(i)}) logP(y_j, Z;\theta) )$$

- M step: find $\theta^{(i+1)}$ to maximize

$$\theta^{(i+1)} = \arg \max_{\theta} Q(\theta, \theta^{(i)})$$

- iterate until convergence
  - criteria: $|\theta^{(i+1)} - \theta^{(i)}| < \epsilon_1$ or $|Q(\theta^{(i+1)}, \theta^{(i)}) - Q(\theta^{(i)}, \theta^{(i)})| < \epsilon_2$


# Hierarchical Clustering

Agglomerative Hierarchical Clustering: merge data point into one cluster
- Compute the proximity matrix
- Let each data point be a cluster
- Repeat: Merge two cloest clusters and updat the proximity matrix
- Until only one single cluster remains

Divisive Hierarchical Clustering: divide one cluster into single data point
- https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec

Simularity metric:
- min linkage $\min Sim(P_i, P_j),  \forall P_i \in C_1, P_j \in C_2$
  - pros: can separate non-elliptical shapes if the gap is not small
  - cons: cannot separate properly if there is noise
- max linkage $\max Sim(P_i, P_j),  \forall P_i \in C_1, P_j \in C_2$
  - pros: does well if there is noise
  - cons: biased towards globular clusters (Ball), tend to break large clusters
- groupby average $\sum Sim(P_i, P_j) / (|C_1| * |C_2|)$
  - pros: does well if there is noise
  - cons: biased towards globular clusters (Ball)
- dsitance between centroids
- ward's method $\sum (dist(P_i, P_j))^2 / (|C_1| * |C_2|)$
  - pros: does well if there is noise
  - cons: biased towards globular clusters (Ball)

# Clustering Evaluation

$$\text{Calinski Harabasz Index} = \frac{SS_B * (N - k)}{SS_W * (k - 1)}$$
- $SS_B$ = between cluster variation
- $SS_W$ = within cluster variation
- $\frac{SS_B}{SS_W}$ should be largest at the optimal clustering size
- http://ethen8181.github.io/machine-learning/clustering_old/clustering/clustering.html